### 1. Reading the datasets
Installing the libraries first.

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
! pip install -q -U numpy==1.19.5
! pip install -q -U matplotlib==3.3.4
! pip install -q -U mxnet==1.8.0
! pip install -q -U catboost==0.26.1
! pip install -q -U lightgbm==3.2.1

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
ERROR: Could not find a version that satisfies the requirement mxnet==1.8.0 (from versions: 1.6.0, 1.7.0.post1, 1.7.0.post2, 1.8.0.post0)
ERROR: No matching distribution found for mxnet==1.8.0
DEPRECATION: Configuring in

In [7]:
import pandas as pd

train_df = pd.read_csv("../final_project/final_project_dataset_training.csv")
test_df = pd.read_csv("../final_project/public_test_features.csv")

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

### 2. Splitting into train and test

__SimpleImputer: Sklearn imputation transformer for completing missing values__

Examples:
* Numerical fields: `SimpleImputer(missing_values=nan, strategy='mean', fill_value=None)` # or strategy='median'
* Categorical fields: `SimpleImputer(missing_values=nan, strategy='most_frequent', fill_value=None)`
* Constant (for both): `SimpleImputer(missing_values=nan, strategy='constant', fill_value=None)` # example: for categorical-> fill_value='missing', for numerical -> fill_value=0

In [9]:
train_df.columns

Index(['ID', 'ASIN', 'marketplace', 'GL', 'category', 'subcategory', 'dept',
       'brand_code', 'brand', 'item_type_keyword', 'website_display_group',
       'product_type_description', 'binding', 'size', 'handling_group',
       'item_count', 'hazmat_classification', 'actual_process_class_ID'],
      dtype='object')

In [10]:
train_df.duplicated().sum() # there is no duplicate

0

In [11]:
columns_to_keep = ['marketplace', 'GL', 'category', 'subcategory', 'brand', 'website_display_group',
       'product_type_description', 'binding','handling_group', 'hazmat_classification']

# dropping columns that have too many missing values such as dept and size, item count
# dropping columns that are too similar to the existing features

In [12]:
test_df.columns

Index(['ID', 'ASIN', 'marketplace', 'GL', 'category', 'subcategory', 'dept',
       'brand_code', 'brand', 'item_type_keyword', 'website_display_group',
       'product_type_description', 'binding', 'size', 'handling_group',
       'item_count', 'hazmat_classification'],
      dtype='object')

In [13]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train_df, test_size=0.1, shuffle=True, random_state=23)

# Print the shapes of the Train - Test Datasets
print('Train - Test Datasets shapes: ', train_data.shape, val_data.shape)

Train - Test Datasets shapes:  (363205, 18) (40357, 18)


In [14]:
train_X=train_data[columns_to_keep]
val_X=val_data[columns_to_keep]
train_Y=train_data['actual_process_class_ID']

In [15]:
train_X.count()/len(train_X)

marketplace                 1.000000
GL                          1.000000
category                    1.000000
subcategory                 1.000000
brand                       0.969885
website_display_group       1.000000
product_type_description    1.000000
binding                     0.994774
handling_group              0.964067
hazmat_classification       0.012599
dtype: float64

### 3. Data Preprocessing and Defining classifiers

Pipeline is an easy way to apply preprocessing on multiple fields.

In [16]:
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [17]:
# Random Forest
# max_features=sqrt(n_features)
random_forest_classifier=RandomForestClassifier(n_estimators=20, criterion='gini',max_depth=None,\
                                                min_samples_split=2,\
                                                min_samples_leaf=1,\
                                                max_leaf_nodes=None,\
                                                min_weight_fraction_leaf=0.0,\
                                                max_features='auto',min_impurity_decrease=0.0,\
                                                min_impurity_split=None,bootstrap=True,oob_score=False,n_jobs=1)

In [18]:
most_frequent_cols = ['brand', 'binding', 'handling_group']
constant_cols = ['hazmat_classification']

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# apply different preprocessing and feature extraction pipelines to different subsets of features, 
# using ColumnTransformer.
most_frequent_transformer = Pipeline(steps=[
       ('cat_imputer', SimpleImputer(strategy='most_frequent'))
])

constant_transformer = Pipeline(steps=[
       ('cat_imputer', SimpleImputer(strategy='constant', fill_value='missing'))
])

preprocessor = ColumnTransformer(
   transformers=[
    ('most_frequent', most_frequent_transformer, most_frequent_cols)
   ,('constant', constant_transformer, constant_cols)
]) 

In [20]:
from sklearn.preprocessing import FunctionTransformer

def to_lowercase(x):
    return np.char.lower(x.astype(str))

lowercase_transformer = FunctionTransformer(to_lowercase, validate=False)

In [21]:
# Preprocess the categorical features and add decision tree to the pipeline
pipeline_decision_tree = Pipeline([('preprocessor', preprocessor),
                                   ('cat_lowercase_transformer', lowercase_transformer),
                                   ('cat_encoder', OneHotEncoder(handle_unknown='ignore')),
                                   ('dt', DecisionTreeClassifier(max_depth=5))])

pipeline_decision_tree.fit(train_X,
                           train_data[["actual_process_class_ID"]])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('most_frequent',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['brand', 'binding',
                                                   'handling_group']),
                                                 ('constant',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant'))]),
                                                  ['hazmat_classification'])])),
                ('cat_lowercase_transformer',
                 FunctionTransformer(func=<function to_lowercase at 0x14fbadb80>)),
 

In [27]:
from sklearn import set_config
set_config(display='diagram')
pipeline_decision_tree

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('most_frequent',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['brand', 'binding',
                                                   'handling_group']),
                                                 ('constant',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant'))]),
                                                  ['hazmat_classification'])])),
                ('cat_lowercase_transformer',
                 FunctionTransformer(func=<function to_lowercase at 0x14fbadb80>)),
                ('cat_encoder', OneHotEncoder(handle_unknown='ignore')),
                ('dt', DecisionTreeClassifier(max_depth=5))])

In [ ]:
from sklearn import set_config
set_config(display='diagram')
pipeline_random_forest_classifier

### 4. Hyperparameter Tuning

In [34]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

hyperparam_grid = {
    'dt__max_depth': [2, 3, 5, 10, 20],
    'dt__min_samples_leaf': [5, 10, 20, 50, 100],
    'dt__criterion': ["gini", "entropy"]
}

hyperparam_search = RandomizedSearchCV(
    pipeline_decision_tree, # base model
    hyperparam_grid,  # hyperparameter grid
    n_iter=100, # number of samples to try 
    # this means we are only picking out 100 combinations of parameters from above to try
    cv=10, # k-fold cross validation
    verbose=1, # print summary
    n_jobs=-1  # use all available processors
)
hyperparam_search

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('most_frequent',
                                                                               Pipeline(steps=[('cat_imputer',
                                                                                                SimpleImputer(strategy='most_frequent'))]),
                                                                               ['brand',
                                                                                'binding',
                                                                                'handling_group']),
                                                                              ('constant',
                                                                               Pipeline(steps=[('cat_imputer',
                                                                                                SimpleImputer(fill_value='missing',
                                                                                                              strategy='constant'))]),
                                                                               ['hazmat_classificat...
                                             ('cat_lowercase_transformer',
                                              FunctionTransformer(func=<function to_lowercase at 0x14fbadb80>)),
                                             ('cat_encoder',
                                              OneHotEncoder(handle_unknown='ignore')),
                                             ('dt',
                                              DecisionTreeClassifier(max_depth=5))]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'dt__criterion': ['gini', 'entropy'],
                                        'dt__max_depth': [2, 3, 5, 10, 20],
                                        'dt__min_samples_leaf': [5, 10, 20, 50,
                                                                 100]},
                   verbose=1)

Fit hyperparameter search to training data

In [ ]:
hyperparam_search.fit(train_X, train_data[["actual_process_class_ID"]])

Fitting 10 folds for each of 50 candidates, totalling 500 fits


In [33]:
# these are the parameters that we can tune
pipeline_decision_tree.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'cat_lowercase_transformer', 'cat_encoder', 'dt', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__most_frequent', 'preprocessor__constant', 'preprocessor__most_frequent__memory', 'preprocessor__most_frequent__steps', 'preprocessor__most_frequent__verbose', 'preprocessor__most_frequent__cat_imputer', 'preprocessor__most_frequent__cat_imputer__add_indicator', 'preprocessor__most_frequent__cat_imputer__copy', 'preprocessor__most_frequent__cat_imputer__fill_value', 'preprocessor__most_frequent__cat_imputer__missing_values', 'preprocessor__most_frequent__cat_imputer__strategy', 'preprocessor__most_frequent__cat_imputer__verbose', 'preprocessor__constant__memory', 'preprocessor__constant__steps', 'preprocessor__constant__verbose', 'preprocessor__constant__cat_imputer', 'preprocessor__constant__

Print the optimal hyperparameters and the corresponding best validation metric

In [ ]:
print(hyperparam_search.best_params_)
print(hyperparam_search.best_score_)

Get the best model out of hyperparameter search and retrain

In [ ]:
pipeline_decision_tree = hyperparam_search.best_estimator_
pipeline_decision_tree.fit(train_X, train_data[["actual_process_class_ID"]])

### 5. Get test predictions and write to CSV for submission

In [ ]:
# call pipeline.predict() on your test dataset
test_predictions_dt=pipeline_decision_tree.predict(test_X)

In [ ]:
import pandas as pd

result_df = pd.DataFrame(columns=["ID",
                                  "actual_process_class_ID"])
result_df["ID"] = test_df["ID"].tolist()
result_df["actual_process_class_ID"] = test_predictions_dt

result_df.to_csv("project_result.csv", index=False)